# How Large Are Transitory and Permanent Shocks?

This notebook illustrates a simple method for measuring the size of transitory and permanent shocks for a set of households whose incomes are subject to those kinds of shocks

In [1]:
# This cell has a bit of initial setup.
# You should substitute whatever setup you might need to address the questions below
import matplotlib.pyplot as plt
import numpy as np
import HARK
from copy import deepcopy
from HARK.utilities import plot_funcs
import statsmodels.api as sm
import statsmodels.formula.api as smf

## Income Process With Transitory and Permanent Shocks

Assume that a household observes two income shocks at the beginning of each period.  Permanent income would be expected to grow by a factor $\Gamma_{t+1}$ in the absence of any shock (this could capture a life cycle profile of income, for example) but that growth is modified by a shock, $\Psi_{t+1}$:
\begin{align}
 P_{t+1} & = \Gamma_{t+1} P_{t}\Psi_{t+1}
\end{align}
whose expected (mean) value is $\mathbb{E}_{t}[\Psi_{t+1}]=1$.  Actual income received $Y$ is equal to permanent income $P$ multiplied by a transitory shock $\Theta$:
\begin{align}
 Y_{t+1} & = P_{t+1}\Theta_{t+1}
\end{align}
where again $\mathbb{E}_{t}[\Theta_{t+1}] = 1$.

For present purposes, we assume that the transitory and permanent shocks are independent.  The transitory shock has two components: A probability $\wp$ that the consumer is unemployed, in which case $\theta^{u}=\theta$, and a probability $(1-\wp)$ of a shock that is a lognormal with a mean chosen so that $\mathbb{E}_{t}[\theta_{t+n}]=1$.

Using lower case variables for the logarithms of all variables above, the process can be written more conveniently as:

\begin{align}
 p_{t+1} & = \gamma_{t+1} + p_{t} + \psi_{t+1}
\\ y_{t+1} & = p_{t+1} + \theta_{t+1}
\end{align}
where both lower-case Greek shocks are now white noise IID variables.

### Removing the Predictable Components

In actual microeconomic data, the first step is to remove the predictable components of income growth (since we are interested here in the shocks).  This is traditionally done by estimating a model of income growth.  Restricting the sample to people of working age, one simply performs a regression:

\begin{align}
 y_{i} & = X_{i} \zeta + \epsilon_{i}
\end{align}

where person $i$ is characterized by observable variables $X_{i}$ in a given period, where $X$ includes usual variables like age, education, and other characteristics known to the person well in advance of the realization of income in the period in question.  The result is an estimate $\hat{\zeta}$ of the consequences of the exogenous variables $X$.  The predictable component of income growth is then obained from
\begin{align}
 \hat{\gamma}_{t,i} & = X_{t,i} \hat{\zeta}
\end{align}

### Estimating The Variances 

It will be notationally convenient to continue using the same variable names as before, where from here on out they are interpreted as having had the predictable component of income growth removed.  That is, we will assume that we now have data generated by a process like:
\begin{align}
 p_{t+1} & = p_{t} + \psi_{t+1}
\\ y_{t+1} & = p_{t+1} + \theta_{t+1}
\end{align}

We can calculate a $d-$period difference in income as
\begin{align}
  y_{t+d} - y_{t} & = p_{t+d} - p_{t} + \theta_{t+d} - \theta_{t}
\\ & =   \psi_{t+1} + \psi_{t+2} + ... \psi_{t+d} + \theta_{t+d} - \theta_{t}
\end{align}
and then we simply take the square of this to obtain:
\begin{align}
  (\Delta^{d}y)^{2} & = \left(\psi_{t+1} + \psi_{t+2} + ... \psi_{t+d} + \theta_{t+d} - \theta_{t}\right)^{2}
\end{align}
but since we have assumed that the $\psi$ and $\theta$ shocks are mutually and serially uncorrelated, the expectation of this simply becomes 
\begin{align}
  \mathbb{E}_{t}[(\Delta^{d}y)^{2}] & = \mathbb{E}_{t}[\psi_{t+1}^{2} + ... + \psi_{t+d}^{2} + 2 \theta^{2}]
\\ & = d \sigma^{2}_{\psi} + 2 \sigma^{2}_\theta
\end{align}

This is an object that we can compute for any individual household.  We can then obtain an estimate of the two objects in question by calculating a set of squared differences in log income for all our households, and performing a very simple regression:

\begin{align}
  (\Delta^{d}y)^{2}_{i} & = \eta_{0} + d \eta_{1}
\end{align}
where $\nu_{1} = \sigma^{2}_{\psi}$ and $\nu_{0} = 2 \sigma^{2}_{\theta}$.


### The Intuition

The intuition for this approach is pretty compelling:  Essentially, the consequence of permanent shocks is to make people get more and more different from each other continually over time as they accumulate permanent shocks.  So the degree to which the _changes_ in income keep increasing and increasing as time passes is a direct measure of the magnitude of the permanent shocks that are causing that divergence.  

If there were no permanent shocks, then in year $t$ and in year $t+d$ the person will have experienced transitory shocks $\theta_{t}$ and $\theta_{t+d}$.  But the very _definition_ of "transitory" means that none of the shocks in intervening periods has any consequence for the difference in incomes between $t$ and $t+d$.  Only the two transitory shocks of those two years contribute.

### Two Small Caveats

#### Time Aggregation

If our data are measured at an annual frequency but income shocks can happen at any time during the year, then even if all shocks were permanent, an exercise like the one above, if conducted using annual data and using only three years of data (so, differences $\Delta^{1}$ and $\Delta^{2}$), would generate a biased answer due to the "time aggregation" problem ([Holbrook Working (1960)](doi:10.2307/1907574Copy); [Edmund Crawley (2020)](https://doi.org/10.1016/j.econlet.2020.108998).  Briefly, imagine two people who are identical up to the beginning of year $t$.  And they both experience identically sized shocks to permanent income in period $t$ -- but one experiences the shock on January 1, and the other on October 1.  Thus, the first person's income is boosted for the entire year by the shock; for this person, the method above would work.  But for the second person, only (1/4) of the year was spent earning the permanently higher level of income.  They would show some increment of income for the year $t$, but then would spend all of year $t+1$ at the higher level of income.  So, there would be a small change in income in year $t$ followed by a (predictable) larger increase in year $t+1$ -- which _looks like_ serial correlation in income growth.  In particular, this problem would manifest itself in a spurious first-order moving average (MA) component in a time series estimation that allowed for such a component.

It can be shown that the problem goes away if only spans of years 2 or more are used.  That is, we have no problem if we use $\Delta^{3}$, $\Delta^{4}$, etc.  Indeed, it can be shown that the magnitude of the permanent shock can be estimated even when there is a _true_ MA component to the transitory shock, simply by restricting the data used to those that span intervals long enough for the MA component to have disappeared.

#### A Lower Bound on $p$

The process above assumes that permanent income can wander to arbitrarily large or small values.  But in practice, all countries that are sophisticated enough to have household datasets of the kind needed to measure these shocks, also have some degree of social insurance that prevents income from falling below some "necessity" level of income.  It can be shown that, if measured income is bounded below (cannot fall too far), then the estimated magnitude of the permanent shocks will be biased down and estimates of the magnitude of the transitory shock will be biased up.

# PROBLEM: Simulate and Estimate 

The $\texttt{ConsIndShockModel}$ consumer type is hard-wired to experience permanent and transitory income shocks exactly like those described above.  

1. Construct a population of `ConsIndShockModel` consumers and simulate them
   * Use the default parameter values 
1. Use the method above to calculate simulated 'empirical' estimates of the magnitude of the shocks
   * Experiment to determine how large a population of consumers, and how long a span of time, you need to correctly recover the 'true' parameter values for the income process

In [2]:
# Choose some calibrated parameters that roughly match steady state 
init_infinite = {
    "CRRA":1.0,                    # Coefficient of relative risk aversion 
    "Rfree":1.01/(1.0 - 1.0/240.0), # Survival probability,
    "PermGroFac":[1.000**0.25], # Permanent income growth factor (no perm growth),
    "PermGroFacAgg":1.0,
    "BoroCnstArt":0.0,
    "CubicBool":False,
    "vFuncBool":False,
    "PermShkStd":[(0.01*4/11)**0.5],  # Standard deviation of permanent shocks to income
    #"PermShkStd":[0.0],  # No permanent shocks to income
    "PermShkCount":7,  # Number of points in permanent income shock grid
    "TranShkStd":[(0.01*4)**0.5],  # Standard deviation of transitory shocks to income,
    #"TranShkStd":[0.0],  # No transitory shocks to income,
    "TranShkCount":5,  # Number of points in transitory income shock grid
    "UnempPrb":0.0,  # Probability of unemployment while working
    "IncUnemp":0.15,  # Unemployment benefit replacement rate
    "UnempPrbRet":0.07,
    "IncUnempRet":0.15,
    "aXtraMin":0.00001,  # Minimum end-of-period assets in grid
    "aXtraMax":20,  # Maximum end-of-period assets in grid
    "aXtraCount":20,  # Number of points in assets grid,
    "aXtraExtra":[None],
    "aXtraNestFac":3,  # Number of times to 'exponentially nest' when constructing assets grid
    #"LivPrb":[1.0 - 1.0/240.0],  # Survival probability
    "LivPrb": [1.0],   #Infinitely living agents
    "DiscFac":0.97,             # Default intertemporal discount factor, # dummy value, will be overwritten
    "cycles":0,
    "T_cycle":1,
    "T_retire":0,
    'T_sim':2000,  # Number of periods to simulate (idiosyncratic shocks model, perpetual youth)
    'T_age':1000,
    'IndL': 10.0/9.0,  # Labor supply per individual (constant),
    'aNrmInitMean':np.log(0.00001),
    'aNrmInitStd':0.0,
    'pLvlInitMean':0.0,
    'pLvlInitStd':0.0,
    'AgentCount':2000,
}

In [3]:
from HARK.ConsumptionSaving.ConsIndShockModel import IndShockConsumerType
ConsumerType = IndShockConsumerType(**init_infinite)

In [4]:
#Solution:
#Controlling the time horizon over which we estimate the transitory and permanent shocks:
ConsumerType.T_sim = 10

#Setting up and simulating the agents' behavior:
ConsumerType.solve(verbose=False)
ConsumerType.track_vars = ['pLvl', 'TranShk', 'PermShk']
ConsumerType.initialize_sim()

ConsumerType.simulate()

SimulatedPermIncome = np.log(ConsumerType.history['pLvl'])
SimulatedTranShocks = np.log(ConsumerType.history['TranShk'])
SimulatedPermShocks = np.log(ConsumerType.history['PermShk'])


SimulatedIncome = SimulatedPermIncome + SimulatedTranShocks
#print(np.shape(SimulatedIncome))   #Just to check the shape of the matrix



#plt.plot(range(ConsumerType.T_sim), SimulatedIncome)
#plt.xlabel('Time t')
#plt.ylabel('Simulated Income')
#plt.show(block=False)

#print(SimulatedPermIncome)

GPFRaw                 = 0.983799 
GPFNrm                 = 0.987147 
GPFAggLivPrb           = 0.983799 
Thorn = APF            = 0.983799 
PermGroFacAdj          = 0.996609 
uInvEpShkuInv          = 1.000000 
VAF                    = 0.970000 
WRPF                   = 0.000000 
DiscFacGPFNrmMax       = 0.982630 
DiscFacGPFAggLivPrbMax = 0.985974 


In [5]:
d = []
y = []
minlag =3

for i in range(ConsumerType.AgentCount):
    for n in range(minlag, ConsumerType.T_sim-1):
        for j in range(ConsumerType.T_sim-n-2):
            Aux1 = (SimulatedIncome[j+n+1,i]- SimulatedIncome[j+1,i])**2
            y.append(Aux1)
            Aux2 = n+1
            d.append(Aux2)
            

x = d
x = sm.add_constant(x)
model = sm.OLS(y,x)
results = model.fit()
results.params

print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     45.25
Date:                Sat, 12 Mar 2022   Prob (F-statistic):           1.76e-11
Time:                        15:42:36   Log-Likelihood:                 23481.
No. Observations:               30000   AIC:                        -4.696e+04
Df Residuals:                   29998   BIC:                        -4.694e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0683      0.003     24.337      0.0

In [6]:
#Comparing our esimation results with the actual standard deviations in the model:
print('The variance of the transitory shock Theta in the model is given by ' + str(ConsumerType.TranShkStd)+ ' where the empirical estimate gives us '+str(np.sqrt(results.params[0]/2)))
print('The variance of the permanent shock Psi in the model is given by ' + str(ConsumerType.PermShkStd)+ ' where the empirical estimate gives us '+str(np.sqrt(results.params[1])))

The variance of the transitory shock Theta in the model is given by [0.2] where the empirical estimate gives us 0.18474536186786522
The variance of the permanent shock Psi in the model is given by [0.06030226891555272] where the empirical estimate gives us 0.05869364112780695


The estimator for the standard deviation of the permanent shock reaped from the coefficient in front of the time difference 'd' is already relativly close to its true value, but the estimation of the transitory shock, garnered from the estimation of the constant in our regression is way too small though. Increasing the agent count did not remedy the problem in a significant way. But increasing the time horizon for teh observations helps tremenduously in overcoming the bias to the constant (i.e. Std of transitory shocks), but it also increased the downward bias to the slope coefficient (i.e. our estimate of Std of permanent shocks).  

My interpretation of this finding is that increasing time, increases the spread in agents permanent income levels, which biases the perception of the slope of the equation. To balance out this bias the constant gets biased in the opposite direction. This is particularly true for small numbers of agents. But I am not entirely satisfied with the result, as it strains against the theory discussed before the statement of the problem. I am looking forward to hear your opinion and see what we did wrong in our code.